In [1]:
def peel_first_col(fname, fcol, delimiter=None):
    with open(fname, 'r') as fin:
        for line in fin:
            try:
                fcol.append(line.split(delimiter, 1)[0])
                yield line.split(delimiter, 1)[1]
            except IndexError:
                continue

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import networkx as nx
import string

symMats = []
graphs = []
algos = []
col = []
algoOrder = ["PoAV", "PQAV", "Po2P", "PQ2P", "PoPN", "PQPN"]
ext =       ["",     "_1",   "_2",   "_3",   "_4",   "_5"]
for e in range(len(ext)):
    for f in range(5):
        col = []
        klDiv = np.loadtxt(peel_first_col("output/testCMs/r"+str(f+1)+ext[e]+".matrix", col), skiprows=1)
        col.pop(0)
        #klDiv = np.absolute(klDiv)
        symKlDiv = np.triu(klDiv)+np.tril(klDiv).T
        symMats = symMats + [symKlDiv]
        
        G = nx.from_numpy_matrix(symKlDiv)
        for i in range(len(col)):
            G.node[i]['id'] = col[i]
        graphs = graphs + [G]
        algos = algos + [algoOrder[e] + str(f)]
print(np.array(col).T)

['m_753.fasta.motif.h1_3.h2_3.h1_1.h1_4.h1_2_chr5.sto.cm'
 'm_9.fasta.motif.h1_4.h2_1.h1_3_chr5.sto.cm'
 'm_1540.fasta.motif.h1_2.h1_4.h1_3_chr5.sto.cm'
 'm_1539.fasta.motif.h2_2.h2_3_chr5.sto.cm'
 'm_1720.fasta.motif.h2_2.h2_5_chr5.sto.cm'
 'm_391.fasta.motif.h2_3.h2_4_chr5.sto.cm'
 'm_417.fasta.motif.h1_5_chr5.sto.cm']


In [3]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters=3).fit(symMats[0])
print(symMats[0])
print(clustering.labels_)

[[  0.     116.451  127.1559 122.7222 135.3996 139.9891 112.9817]
 [  0.       0.     110.1554 115.6374 130.0208 135.8775  99.9079]
 [  0.       0.       0.      12.0407 130.8657 138.1505 103.7424]
 [  0.       0.       0.       0.     129.3254 137.6009 104.5831]
 [  0.       0.       0.       0.       0.     148.3967 118.2296]
 [  0.       0.       0.       0.       0.       0.     117.2208]
 [  0.       0.       0.       0.       0.       0.       0.    ]]
[1 1 0 0 0 2 2]


In [7]:
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn import metrics

for i in range(len(symMats)):
    adj_mat = symMats[i] + symMats[i].T
    aff_mat = np.exp(-1 * adj_mat / adj_mat.std())

    sc = SpectralClustering(3, affinity='precomputed', n_init=100)
    sc.fit(aff_mat)
    for n in range(len(sc.labels_)):
        graphs[i].node[n]['group'] = str(sc.labels_[n])
    
    print(str(algos[i]), "\t", sc.labels_)
    if (i % 5 == 4):
        print()

PoAV0 	 [1 1 2 2 1 0 1]
PoAV1 	 [1 1 0 0 2 1 1]
PoAV2 	 [0 2 1 1 2 0 2]
PoAV3 	 [1 1 0 0 1 2 1]
PoAV4 	 [0 0 1 1 0 2 2]

PQAV0 	 [0 0 1 0 2 2 1]
PQAV1 	 [1 1 1 0 0 2 2]
PQAV2 	 [1 2 2 0 0 1 0]
PQAV3 	 [0 1 1 2 1 2 0]
PQAV4 	 [2 1 1 0 1 0 2]

Po2P0 	 [1 1 0 0 1 2 1]
Po2P1 	 [2 1 0 0 1 2 1]
Po2P2 	 [1 0 2 2 0 1 0]
Po2P3 	 [0 0 1 1 0 2 0]
Po2P4 	 [2 2 0 0 2 1 2]

PQ2P0 	 [1 1 0 0 1 2 1]
PQ2P1 	 [2 0 1 1 0 2 0]
PQ2P2 	 [2 0 1 1 0 2 0]
PQ2P3 	 [0 0 1 1 0 2 0]
PQ2P4 	 [2 2 1 1 2 0 2]

PoPN0 	 [2 2 1 1 0 0 2]
PoPN1 	 [2 1 0 0 2 1 1]
PoPN2 	 [0 0 1 1 0 2 0]
PoPN3 	 [1 1 0 0 1 2 1]
PoPN4 	 [2 2 1 1 0 2 2]

PQPN0 	 [2 2 1 1 0 0 1]
PQPN1 	 [2 0 1 1 2 0 0]
PQPN2 	 [2 0 1 1 0 0 2]
PQPN3 	 [1 1 2 0 1 1 0]
PQPN4 	 [0 0 1 1 2 0 0]



In [5]:
for i in range(len(graphs)):
    G=graphs[i]
    nx.write_gexf(G,"output/"+str(algos[i])+".gexf")